In [21]:
import numpy as np
import pandas as pd
import io
import requests

In [22]:
url="https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/ecdc/full_data.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0.0,0,0.0,0
1,2020-01-01,Afghanistan,0.0,0,0.0,0
2,2020-01-02,Afghanistan,0.0,0,0.0,0
3,2020-01-03,Afghanistan,0.0,0,0.0,0
4,2020-01-04,Afghanistan,0.0,0,0.0,0


In [23]:
def ProcessData(covid_path, Map_path):
    Map=pd.read_csv(Map_path)
    df.reset_index(level=0, inplace=True)
    df1=df.groupby('location').max()
    list1 = df1['index'].tolist()
    covid=df.loc[df['index'].isin(list1)]
    data=pd.merge(Map, covid,on=['location'], how='left')
    data['MapID']=data['MapID'].map('{:0>3}'.format)
   
    return data
data=ProcessData(df,'ProcessedTSV.csv')
data.tail()

,MapID,Code,location,name_long,continent,economy,income_grp,index,date,new_cases,new_deaths,total_cases,total_deaths
236,732,SAH,W. Sahara,Western Sahara,Africa,7. Least developed region,5. Low income,NaN,NaN,NaN,NaN,NaN,NaN
237,876,WLF,Wallis and Futuna Is.,Wallis and Futuna Islands,Oceania,6. Developing region,4. Lower middle income,NaN,NaN,NaN,NaN,NaN,NaN
238,887,YEM,Yemen,Yemen,Asia,7. Least developed region,4. Lower middle income,13458.0,2020-04-27,0.0,0.0,1.0,0.0
239,894,ZMB,Zambia,Zambia,Africa,7. Least developed region,4. Lower middle income,13498.0,2020-04-27,4.0,0.0,88.0,3.0
240,716,ZWE,Zimbabwe,Zimbabwe,Africa,5. Emerging region: G20,5. Low income,13536.0,2020-04-27,0.0,0.0,31.0,4.0


In [24]:
data.date.unique()

array(['2020-04-27', nan], dtype=object)

In [25]:
bins = [ -1, 50, 100, 500, 1000, 5000, 10000, 20000,30000, np.inf]
names = [    50, 100, 500 ,1000, 5000, 10000, 20000, 30000, 600000]

data['Catog'] = pd.cut(data['total_deaths'], bins, labels=names)

In [26]:
sorted(data.Catog.unique())

[100, nan, 50, 500, 1000, 5000, 10000, 30000, 600000]

In [27]:
data['Catog'] = data['Catog'].astype(str)

In [28]:
data.to_csv('CovidData.tsv', sep = '\t')